In [1]:
import kagglehub
import pandas as pd
import numpy as np
 # Download latest version
path = kagglehub.dataset_download("wenruliu/adult-income-dataset")

print("Path to dataset files:", path)

c:\Users\Usuario\Downloads\Automações\Privacidade_Diferencial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Usuario\.cache\kagglehub\datasets\wenruliu\adult-income-dataset\versions\2


In [5]:
df = pd.read_csv(fr'{path}\adult.csv')

def prepocessing(df):

    df.drop(columns=['fnlwgt', 'education', 'capital-gain', 'capital-loss', 'hours-per-week'], inplace = True)

    fill_rows = ~(df == '?').any(axis = 1)
    df = df[fill_rows]
    
    for col in df.columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
          df[col], _ = pd.factorize(df[col])

    return df

def split_train_test(df, test_size=0.3):
    # 1. Embaralha o DataFrame inteiro aleatoriamente
    # Usa sample(frac=1) para selecionar 100% dos dados de forma aleatória
    # random_state é usado para reprodutibilidade (sempre a mesma ordem se o número for o mesmo)
    shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # 2. Calcula o ponto de corte
    len_df = len(shuffled_df)
    cut_point = len_df - round(len_df * test_size)
    
    # 3. Divide o DataFrame embaralhado
    train = shuffled_df[:cut_point]
    test = shuffled_df[cut_point:]
    
    print(f'Tamanho total: {len(df)}')
    print(f'Tamanho do treino: {len(train)}')
    print(f'Tamanho do teste: {len(test)}')
    
    return train, test

df = prepocessing(df)
X_train, X_test = split_train_test(df)

Tamanho total: 45222
Tamanho do treino: 31655
Tamanho do teste: 13567


C:\Users\Usuario\AppData\Local\Temp\ipykernel_22940\1569553445.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
C:\Users\Usuario\AppData\Local\Temp\ipykernel_22940\1569553445.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col], _ = pd.factorize(df[col])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_22940\1569553445.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
C:\Users\Usuari

In [ ]:
import numpy as np
import pandas as pd
import time

class KNN_Otimizado():
    def __init__(self, k):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        # Converter para float32 acelera contas e economiza memória
        self.X_train = np.array(X, dtype=np.float32)
        self.y_train = np.array(y)

    def get_vizinhos_indices(self, test_row):
        # --- OTIMIZAÇÃO PRINCIPAL ---
        # 1. Subtração Broadcasting: (Matriz - Vetor)
        # O numpy subtrai 'test_row' de todas as linhas de X_train simultaneamente
        diferencas = self.X_train - test_row
        
        # 2. Quadrado e Soma
        distancias_sq = np.sum(diferencas**2, axis=1)
        # Nota: Não precisamos tirar a raiz quadrada (np.sqrt) para ordenar,
        # pois se x^2 > y^2, então x > y (para distâncias positivas).
        # Isso economiza uma operação pesada.
        
        # 3. Ordenação
        # np.argsort retorna os ÍNDICES dos elementos ordenados
        indices_ordenados = np.argsort(distancias_sq)
        
        # Retorna os índices dos k primeiros
        return indices_ordenados[:self.k]

    def predict_instance(self, test_row):
        indices = self.get_vizinhos_indices(test_row)
        
        # Pega as classes correspondentes aos índices
        classes_vizinhas = self.y_train[indices]
        
        # Votação majoritária
        valores, contagens = np.unique(classes_vizinhas, return_counts=True)
        indice_vencedor = np.argmax(contagens)
        return valores[indice_vencedor]
  
    def predict(self, X_test):
        X_test = np.array(X_test, dtype=np.float32)
        predicoes = []
        total = len(X_test)
        
        start_time = time.time()
        
        for i, row in enumerate(X_test):
            pred = self.predict_instance(row)
            predicoes.append(pred)
            
            # Print de progresso a cada 100 iterações para não poluir o terminal
            if i % 100 == 0:
                percent = (i / total) * 100
                elapsed = time.time() - start_time
                print(f"Progresso: {percent:.1f}% | Linha {i}/{total} | Tempo: {elapsed:.0f}s", end='\r')
                
        print(f"\nConcluído em {time.time() - start_time:.0f} segundos.")
        return np.array(predicoes)

# -----------------------------------------------------------
# PREPARAÇÃO DOS DADOS (Ajustado para evitar sobrescrita de variáveis)
# -----------------------------------------------------------

# Supondo que X_train e X_test sejam seus DataFrames originais carregados
# Vamos usar variaveis auxiliares para não perder os dados originais
y_treino_arr = X_train['age'].values
X_treino_arr = X_train.drop(columns=['age']).values

y_teste_arr = X_test['age'].values
X_teste_arr = X_test.drop(columns=['age']).values

k = 10 
model = KNN_Otimizado(k=k)

# Treinamento
print("Treinando modelo...")
model.fit(X_treino_arr, y_treino_arr)

# Execução
print("Executando KNN Otimizado (previsão)...")
y_pred_padrao = model.predict(X_teste_arr)

# Salvando
df_resultado = pd.DataFrame({
    'y_true': y_teste_arr,
    'y_pred': y_pred_padrao
})
df_resultado.to_csv('resultado_knn_tradicional.csv', index=False)
print("Resultado Tradicional salvo.")

Treinando modelo...
Executando KNN Tradicional...


In [ ]:
def acuracia(pred, y_target):
    return  sum(pred == y_target)/len(y_target)


,age,workclass,educational-num,marital-status,occupation,relationship,race,gender,native-country,income
31655,47,0,9,4,6,3,1,1,0,0
31656,40,4,13,1,4,4,0,1,0,0
31657,30,0,13,0,9,2,1,0,0,0
31658,19,0,10,0,3,5,1,1,0,0
31659,58,2,10,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
45217,39,2,6,3,5,2,0,0,0,0
45218,36,1,14,1,4,1,1,0,0,1
45219,53,1,5,1,7,1,1,0,0,0
45220,57,0,11,0,3,0,1,1,0,0
